In [1]:
from __future__ import print_function
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from data import ModelNet40
from model import PointNet, DGCNN_cls
import numpy as np
from torch.utils.data import DataLoader
from util import cal_loss, IOStream, vis_point_cloud, vis_point_cloud_side_by_side
import sklearn.metrics as metrics
import pickle
import open3d as o3d

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class args:
    def __init__(self):
        self.batch_size = 32
        self.num_points = 1024
        self.no_cuda = not torch.cuda.is_available()
        self.cuda = torch.cuda.is_available()
        self.seed = 1
        self.exp_name = "cls_1024_eval"
        self.model = "dgcnn"
        self.dataset = "modelnet40"
        self.test_batch_size = 16
        self.epochs = 50 #original 250
        self.lr = 0.001
        self.momentum = 0.9
        self.scheduler = "cos"
        self.dropout = 0.5
        self.k = 20
        self.model_path = "pretrained/model.cls.1024.t7"
        self.emb_dims = 1024
args = args()

if not os.path.exists("outputs"):
    os.makedirs("outputs")
if not os.path.exists("outputs/" + args.exp_name):
    os.makedirs("outputs/" + args.exp_name)
if not os.path.exists("outputs/" + args.exp_name + "/" + "models"):
    os.makedirs("outputs/" + args.exp_name + "/" + "models")
os.system(
    "cp main_cls.py outputs" + "/" + args.exp_name + "/" + "main_cls.py.backup"
)
os.system("cp model.py outputs" + "/" + args.exp_name + "/" + "model.py.backup")
os.system("cp util.py outputs" + "/" + args.exp_name + "/" + "util.py.backup")
os.system("cp data.py outputs" + "/" + args.exp_name + "/" + "data.py.backup")
print(device)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
cuda


In [8]:
def test(model_path):
    #sample test
    test_loader = DataLoader(
            ModelNet40(partition="test", num_points=args.num_points),
            batch_size=args.test_batch_size,
            shuffle=True,
            drop_last=False,
        )

    # Try to load models
    if args.model == "pointnet":
        model = PointNet(args).to(device)
    elif args.model == "dgcnn":
        model = DGCNN_cls(args).to(device)
    else:
        raise Exception("Not implemented")

    model = nn.DataParallel(model)
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model = model.to(device).eval()
    test_acc = 0.0
    count = 0.0
    test_true = []
    test_pred = []
    for data, label in test_loader:
        count += 1
        # print(count / len(test_loader))
        data, label = data.to(device), label.to(device).squeeze()
        data = data.permute(0, 2, 1)
        batch_size = data.size()[0]
        logits = model(data)
        preds = logits.max(dim=1)[1]
        test_true.append(label.cpu().numpy())
        test_pred.append(preds.detach().cpu().numpy())
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    test_acc = metrics.accuracy_score(test_true, test_pred)
    avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
    outstr = "Test :: test acc: %.6f, test avg acc: %.6f" % (
        test_acc,
        avg_per_class_acc,
    )
    print(outstr)

In [4]:
#train
train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=True,
    )
test_loader = DataLoader(
    ModelNet40(partition="test", num_points=args.num_points),
    num_workers=8,
    batch_size=args.test_batch_size,
    shuffle=True,
    drop_last=False,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Try to load models
if args.model == "pointnet":
    model = PointNet(args).to(device)
elif args.model == "dgcnn":
    model = DGCNN_cls(args).to(device)
else:
    raise Exception("Not implemented")

# print(str(model))

model = nn.DataParallel(model)
print("Let's use", torch.cuda.device_count(), "GPUs!")

opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)

if args.scheduler == "cos":
    scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-3)
elif args.scheduler == "step":
    scheduler = StepLR(opt, step_size=20, gamma=0.7)

criterion = cal_loss
best_test_acc = 0

EPSILON = 0.2
epsilons = [0.02, 0.05] #omitted 0.1
accuracies = {}

for e in epsilons:
    print("Epsilon: ", e)
    model = DGCNN_cls(args).to(device)
    model = nn.DataParallel(model)
    opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)
    if args.scheduler == "cos":
        scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-3)
    elif args.scheduler == "step":
        scheduler = StepLR(opt, step_size=20, gamma=0.7)

    for epoch in range(args.epochs):
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data, label in train_loader:
            data, label = data.to(device), label.to(device).squeeze()
            data = data.permute(0, 2, 1)
            data.requires_grad = True
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            train_loss += loss.item() * batch_size
            vis_data = data.permute(0, 2, 1).cpu().detach().numpy()

            #perform fgsm
            data_grad = data.grad.data
            perturbed = data + e * data_grad.sign()
            vis_data_perturbed = perturbed.permute(0, 2, 1).cpu().detach().numpy()

            # vis_point_cloud_side_by_side(vis_data[0], vis_data_perturbed[0])
            
            #training model with fgsm data
            opt.zero_grad()
            perturbed_logits = model(perturbed)
            loss = criterion(perturbed_logits, label)
            loss.backward()
            opt.step()
            train_loss += loss.item() * batch_size

            #calculate accuracy
            preds = logits.max(dim=1)[1]
            preds_perturbed = perturbed_logits.max(dim=1)[1]
            count += batch_size * 2
            train_true.append(label.cpu().numpy())
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())
            train_pred.append(preds_perturbed.detach().cpu().numpy())
            
        if args.scheduler == "cos":
            scheduler.step()
        elif args.scheduler == "step":
            if opt.param_groups[0]["lr"] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]["lr"] < 1e-5:
                for param_group in opt.param_groups:
                    param_group["lr"] = 1e-5
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        outstr = "\tEpoch %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f" % (
            epoch,
            train_loss * 1.0 / count,
            metrics.accuracy_score(train_true, train_pred),
            metrics.balanced_accuracy_score(train_true, train_pred),
        )
        print(outstr)
        accuracies[e] = metrics.accuracy_score(train_true, train_pred)
    path = "./izzymodels/"+str(e)[2:] + ".pth"
    torch.save(model.state_dict(), path)
accuracies

Let's use 2 GPUs!
Epsilon:  0.02
	Epoch 0, loss: 1.731565, train acc: 0.524684, train avg acc: 0.344497
	Epoch 1, loss: 1.033640, train acc: 0.682868, train avg acc: 0.545623
	Epoch 2, loss: 0.865629, train acc: 0.729642, train avg acc: 0.610029
	Epoch 3, loss: 0.775295, train acc: 0.754733, train avg acc: 0.644382
	Epoch 4, loss: 0.709548, train acc: 0.776517, train avg acc: 0.680484
	Epoch 5, loss: 0.655448, train acc: 0.794076, train avg acc: 0.699314
	Epoch 6, loss: 0.636687, train acc: 0.797180, train avg acc: 0.703190
	Epoch 7, loss: 0.601078, train acc: 0.807360, train avg acc: 0.719908
	Epoch 8, loss: 0.566458, train acc: 0.817182, train avg acc: 0.733766
	Epoch 9, loss: 0.549937, train acc: 0.821967, train avg acc: 0.739701
	Epoch 10, loss: 0.539849, train acc: 0.823290, train avg acc: 0.742100
	Epoch 11, loss: 0.522736, train acc: 0.831179, train avg acc: 0.751075
	Epoch 12, loss: 0.510825, train acc: 0.836981, train avg acc: 0.762513
	Epoch 13, loss: 0.493968, train acc: 0.8

KeyboardInterrupt: 

In [7]:
mod = DGCNN_cls(args).to(device)
mod = nn.DataParallel(mod)
mod.load_state_dict(torch.load(args.model_path, map_location=torch.device(device)))

train_loader = DataLoader(
        ModelNet40(partition="train", num_points=args.num_points),
        num_workers=8,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
    )
print("getting sample")
sample = None
label = None
for data, l in train_loader:
    print(0)
    data, l = data.to(device), l.to(device).squeeze()
    sample, label = data, l
    break
vis_point_cloud(sample[0].cpu())

getting sample
0


In [5]:
accuracies

{0.02: 0.8888436482084691, 0.05: 0.7661339576547231}

In [9]:
model_path = "./izzymodels/02.pth"
test(model_path)

Test :: test acc: 0.891005, test avg acc: 0.841599
